# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,anadyr,64.7500,177.4833,-25.72,84,0,4.00,RU,1710702561
1,1,olyokminsk,60.3736,120.4264,-23.37,100,92,0.81,RU,1710702561
2,2,college,64.8569,-147.8028,-11.01,64,100,2.57,US,1710702561
3,3,punta arenas,-53.1500,-70.9167,8.06,53,75,11.83,CL,1710702351
4,4,vernon,50.2581,-119.2691,15.42,50,0,1.03,CA,1710702312


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [13]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    'Lng',  # Change to the correct longitude column name
    'Lat',  # Change to the correct latitude column name
    size='Humidity',
    # hover_cols=['City', 'Humidity'],
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=400,
    title='City Humidity Levels',
    color='City',
    scale = .75,
    alpha=0.4  # Adjust transparency for better visibility
)
# Display the map
humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp" ] < 27) & (city_data_df["Max Temp"] > 21)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,kalmunai,7.4167,81.8167,24.70,78,0,0.58,LK,1710702577
114,114,holualoa,19.6228,-155.9522,23.21,77,0,2.06,US,1710702583
148,148,port blair,11.6667,92.7500,25.66,74,0,2.22,IN,1710702482
194,194,laguna,38.4210,-121.4238,21.41,59,0,0.89,US,1710702363
276,276,molina,-35.1167,-71.2833,24.85,31,0,3.60,CL,1710702623
296,296,castillos,-34.1667,-53.8333,22.89,53,0,4.08,UY,1710702628
353,353,zagora,30.3484,-5.8365,26.36,11,0,4.46,MA,1710702639
393,393,ghadamis,30.1337,9.5007,21.02,27,0,3.34,LY,1710702647
436,436,douz,33.4663,9.0203,21.80,29,0,3.21,TN,1710702656
446,446,kailua-kona,19.6406,-155.9956,25.28,76,0,2.06,US,1710702658


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.loc[:,['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
79,kalmunai,LK,7.4167,81.8167,78,
114,holualoa,US,19.6228,-155.9522,77,
148,port blair,IN,11.6667,92.7500,74,
194,laguna,US,38.4210,-121.4238,59,
276,molina,CL,-35.1167,-71.2833,31,
296,castillos,UY,-34.1667,-53.8333,53,
353,zagora,MA,30.3484,-5.8365,11,
393,ghadamis,LY,30.1337,9.5007,27,
436,douz,TN,33.4663,9.0203,29,
446,kailua-kona,US,19.6406,-155.9956,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {'apiKey':geoapify_key, 
          'categories' : 'accommodation.hotel'
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row ['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kalmunai - nearest hotel: Saji new food
holualoa - nearest hotel: Kona Hotel
port blair - nearest hotel: Andaman Ocean Breeze
laguna - nearest hotel: Holiday Inn Express & Suites
molina - nearest hotel: La Cabaña
castillos - nearest hotel: A mi gente
zagora - nearest hotel: Riad Le Jasmin Du Sud
ghadamis - nearest hotel: فندق عين الفارس
douz - nearest hotel: Louage station Douz
kailua-kona - nearest hotel: Kona Seaside Hotel
bamboo flat - nearest hotel: Hotel Megapode Nest
diu - nearest hotel: Hotel Ankur
kindu - nearest hotel: Hôtel Karibu


,City,Country,Lat,Lng,Humidity,Hotel Name
79,kalmunai,LK,7.4167,81.8167,78,Saji new food
114,holualoa,US,19.6228,-155.9522,77,Kona Hotel
148,port blair,IN,11.6667,92.7500,74,Andaman Ocean Breeze
194,laguna,US,38.4210,-121.4238,59,Holiday Inn Express & Suites
276,molina,CL,-35.1167,-71.2833,31,La Cabaña
296,castillos,UY,-34.1667,-53.8333,53,A mi gente
353,zagora,MA,30.3484,-5.8365,11,Riad Le Jasmin Du Sud
393,ghadamis,LY,30.1337,9.5007,27,فندق عين الفارس
436,douz,TN,33.4663,9.0203,29,Louage station Douz
446,kailua-kona,US,19.6406,-155.9956,76,Kona Seaside Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 
    'Lat',  
    size='Humidity',
    hover_cols=['Country', 'Hotel Name'],
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=400,
    title='Hotel Name and the Country',
    color='City',
    scale = .75,
    alpha=0.9  # Adjust transparency for better visibility
)
# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)